<a href="https://colab.research.google.com/github/cemvardar/pragmatic_google_colab/blob/main/dslab_colab_utility_image_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import os
file_path = 'dslab_colab_utility_main.ipynb'
if os.path.exists(file_path):
    os.remove(file_path)


!wget https://github.com/cemvardar/pragmatic_google_colab/raw/main/dslab_colab_utility_main.ipynb
%run dslab_colab_utility_main.ipynb

In [2]:
from google.colab import userdata
from PIL import Image as Image_PIL
from io import BytesIO
import io
from dataclasses import dataclass

In [3]:
@dataclass
class LiplipsImage:
    url: str
    id: str
    content: str=None
    local_path: str=None
    gcp_path: str=None
    alt_text: str=None
    description: str=None
    title: str=None




def reduce_image_size(image_content, size_reduction_factor = 0.5):
    img = Image_PIL.open(BytesIO(image_content))

    # Calculate new dimensions
    new_width = int(img.width * size_reduction_factor)
    new_height = int(img.height * size_reduction_factor)

    # Resize the image
    img = img.resize((new_width, new_height), Image_PIL.LANCZOS)
    byte_arr = io.BytesIO()
    img.save(byte_arr, format='PNG')
    byte_arr = byte_arr.getvalue()
    return byte_arr


In [4]:
def create_ai_image_with_guide(id, image_url, guidence_image_url, seed, sub_folder_path):

    # Define the URL and parameters
    url = 'https://image-api.photoroom.com/v2/edit'
    params = {
        # 'background.prompt': prompt,
        'background.guidance.imageUrl': guidence_image_url,
        'background.seed': seed,
        'padding': '0.1',
        'shadow.mode': 'ai.soft',
        'imageUrl': image_url
    }

    headers = {
        # 'x-api-key': userdata.get('photo_room_live'),
        'x-api-key': userdata.get('photo_room_sandbox'),
        'pr-ai-background-model-version': '3'
    }

    # Make the GET request
    response = requests.get(url, headers=headers, params=params)

    image_id = f"{id}_{get_time_stamp_string()}"
    image_file_name = f"{id}_seed_{seed}_{prompt.replace(' ', '-')}"
    image_path = get_file_name_for_export_with_date_time(f'{image_file_name}', 'png')
    with open(image_path, 'wb') as f:
        f.write(response.content)
    uploaded_file_gcp_link = upload_file_to_gcp_generic_mime_type(image_path, sub_folder_path)

    rs_image = LiplipsImage(id=image_id, url=uploaded_file_gcp_link)
    rs_image.content = reduce_image_size(response.content, 0.2)
    rs_image.local_path = image_path
    rs_image.gcp_path = uploaded_file_gcp_link
    return rs_image

In [5]:
def create_ai_image(id, image_url, prompt, seed, sub_folder_path,
                    remove_watermark=False, image_file_name=None,
                    expandPrompt=None, negative_prompt=None):
    # Define the URL and parameters
    url = 'https://image-api.photoroom.com/v2/edit'
    params = {
        'background.prompt': prompt,
        'background.seed': seed,
        'background.expandPrompt': expandPrompt,
        'background.negativePrompt': negative_prompt,
        'padding': '0.1',
        'shadow.mode': 'ai.soft',
        'imageUrl': image_url
    }

    if remove_watermark:
        api_key = userdata.get('photo_room_live')
    else:
        api_key = userdata.get('photo_room_sandbox')


    headers = {
        'x-api-key': api_key,
        'pr-ai-background-model-version': '3'
    }

    # Make the GET request
    response = requests.get(url, headers=headers, params=params)
    if image_file_name is None:
        image_id = f"{id}_{get_time_stamp_string()}"
        image_file_name = f"{id}_seed_{seed}_{prompt[:50].replace(' ', '-')}"
        image_file_name = get_file_name_for_export_with_date_time(f'{image_file_name}', 'png')
    with open(image_path, 'wb') as f:
        f.write(response.content)
    uploaded_file_gcp_link = upload_file_to_gcp_generic_mime_type(image_file_name, sub_folder_path)

    rs_image = LiplipsImage(id=image_id, url=uploaded_file_gcp_link)
    rs_image.content = reduce_image_size(response.content, 0.2)
    rs_image.local_path = image_file_name
    rs_image.gcp_path = uploaded_file_gcp_link
    return rs_image

In [ ]:
# prompts = [
# # 'surface light white without shade',
# # 'on a full marble',
# # 'on a white surface with 1 plant',
# 'on a white surface with 1 flower',
# # 'on a wood table with 1 succulant',
# 'on a white wood surface with small stones',
# # 'on a white wood table with 1 dry flower',
# # 'on a white wood table with small quartz gemstones',
# # 'on a white wood surface with 1 small seashell',
# # 'on a white fabric on table',
# # 'the image displays a necklace on a table with a beautiful orange yellow sunset where the clouds are brightly lit from the sunset and a field is in the background slightly blurry',
# # 'on a white wood table with 1 small seashell',
# # 'on a soft baby blue surface'
#  ]

In [ ]:
# id = '2581'
# url = 'https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581.jpg'
# sub_folder_path = 'liplips/roma_silvers_products/rs2502'
# seeds = [117879368, 55994449, 48672244, 65080068]
# remove_watermark = False
# ai_image = create_ai_image(id, url, prompts[0], seeds[0], sub_folder_path, remove_watermark)

https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581_seed_117879368_on-a-white-surface-with-1-flower_07_21_2024_10_33_55.png


In [ ]:
# guidence_image_url = 'https://www.liplips.co.uk/cdn/shop/files/2541_prompt_name_pink_background_seed_117879368_07_10_2024_17_18_03_1024x1024@2x.png?v=1720652582'
# guidence_image_url = 'https://www.liplips.co.uk/cdn/shop/files/phuket-ametist-balik-kolye-rs2281-1925-Photoroom_1024x1024@2x.jpg'
# guidence_image_url = 'https://www.liplips.co.uk/cdn/shop/files/phuket-apatit-balik-kolye-rs2280-2365-Photoroom_2_1024x1024@2x.jpg?v=1720995071'
# image = create_ai_image_with_guide(id, url, guidence_image_url, seeds[0], sub_folder_path)

https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581_seed_117879368_guidence_image_07_21_2024_10_34_07.png


In [ ]:
def create_liplips_image_from_url(url: str, id: str, download_folder: str='/content') -> LiplipsImage:
    # Create the download folder if it doesn't exist
    os.makedirs(download_folder, exist_ok=True)

    # Download the image
    response = requests.get(url)

    if response.status_code == 200:
        # Determine the local path
        local_filename = os.path.join(download_folder, f"{id}.jpg")

        # Save the image locally
        with open(local_filename, 'wb') as f:
            f.write(response.content)

        # Create LiplipsImage object
        liplips_image = LiplipsImage(
            url=url,
            id=id,
            content=response.content,
            local_path=local_filename,
            gcp_path=url
        )

        return liplips_image
    else:
        raise Exception(f"Failed to download image from {url}, status code: {response.status_code}")

# Example usage
# url = 'https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs2502/2581.jpg'
# id = '2581'
# image = create_liplips_image_from_url(url, id)


'07_27_2024_12_02_13'

In [ ]:
# "A necklace is nicely placed on a pristine white wood table. The table is adorned with a single dried flower, adding a touch of rustic charm. Soft natural light filters through a nearby window, casting a gentle shadow of the pants onto the table surface, creating a serene and minimalist composition",
# prompt = "A delicate necklace with an elegant pendant, placed gracefully on a rustic white wood table. The necklace should have a subtle shine, reflecting the natural light softly. Beside the necklace, include a single, artistically positioned dry flower, such as a lavender sprig or a small dried rose, adding a touch of vintage charm. The background should be minimalist, with the focus clearly on the necklace and the flower. Ensure the overall composition exudes a sense of elegance and simplicity, with the textures of the wood, the metallic shine of the necklace, and the delicate nature of the dry flower all harmoniously balanced."
# prompt = "Place the delicate necklace with its elegant pendant and pearl accents on a rustic white wood table. Ensure the table's texture is visible to add a touch of natural charm. Position a single dry flower, such as a lavender sprig or a small dried rose, next to the necklace, creating a simple yet sophisticated composition. Use soft, natural lighting that casts gentle shadows and highlights the intricate details of the necklace and the flower. The overall scene should have a calm, minimalist aesthetic, with a clean background that keeps the focus on the necklace. The colors should be muted, with the warm tones of the gold and the soft hues of the pearls standing out subtly against the white wood. The dry flower should add a hint of vintage elegance without overpowering the main subject."
# prompt = "A delicate necklace with gold and pearl accents on a white wood table. The table features a single dry flower beside the necklace, adding a touch of elegance. Soft natural lighting streams from the left, casting gentle shadows and highlighting the intricate details of the necklace. The background is a clean, minimalist white with a hint of texture to enhance the sophistication of the photo. The entire scene is framed in a way that emphasizes the necklace's beauty and craftsmanship, creating a professional yet cute atmosphere."

In [9]:
# # example image generation
# gcp_path = 'https://storage.googleapis.com/decision-science-lab-bucket/liplips/roma_silvers_products/rs1236/929.jpg'
# sub_folder_path = 'liplips/shopify_liplips_images_ai/test'
# remove_watermark = False
# # prompt = 'a person posing for a picture on a lawn, with a flower garden in the background'
# prompt = 'the image displays a jewellery piece with succulents in the background. The image features a minimalistic and professional looking background. '
# seeds = [117879368, 55994449, 48672244, 65080068]
# for seed in seeds:
#     image = create_ai_image('test', gcp_path, prompt, seed, sub_folder_path, False, "never")

https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_117879368_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_01_20.png
https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_55994449_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_01_29.png
https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_48672244_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_01_34.png
https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_65080068_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_01_38.png


In [10]:
# for seed in seeds:
#     image = create_ai_image('test', gcp_path, prompt, seed, sub_folder_path, False, "auto")

https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_117879368_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_02_04.png
https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_55994449_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_02_08.png
https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_48672244_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_02_15.png
https://storage.googleapis.com/decision-science-lab-bucket/liplips/shopify_liplips_images_ai/test/test_seed_65080068_the-image-displays-a-jewellery-piece-with-succulen_08_29_2024_12_02_21.png
